# Use OpenAI Gym environments from C++


> "how to interact with an OpenAI Gym environment from C++ "

- toc:true
- branch: master
- badges: false
- comments: false
- author: Alexandros Giavaras
- categories: [programming, OpenAI-Gym, reinforcement-learning, C++, Python, boost-python]

## <a name="overview"></a> Overview

OpenAI-Gym is one of the most commonly used of Python packages used when developing reinforcement learning algorithms. In this post, we use the ```boost::python``` library to interact with an OpenAI-Gym environment  from a C++ program. 

## Use OpenAI Gym environments from C++

In this post we use the <a href="https://www.boost.org/doc/libs/1_76_0/libs/python/doc/html/index.html">boost::python</a> library in order to interact with an OpenAI-Gym environment. Specifically, we will interact with the ```FrozenLake-v0``` environment. The exposition here is meant to be minimal rather than exhaustive.

In order to use ```boost::python``` we need to include it.

```
#include <boost/python.hpp>
```

Before starting the interaction with the Python interpreter, we need to initialize it. This is done by calling  

```
Py_Initialize()
```

We next import the module of interest that is ```gym```. We obtain a ```namespace``` so that we can use it to obtain the created environment in the interpreter. This is done in the following line

```
auto gym_namespace = gym_module.attr("__dict__");
```

We can extract various attributes. For instance, the module name as shown below

```
std::cout<<"Module name "<<boost::python::extract<const char*>(gym_namespace["__name__"])<<std::endl;
```

The line of interest is where we create the environment in the interpreter and get a pointer back in the C++ program

```
 // create an environment
auto ignored = boost::python::exec("import gym \n"
                                   "world = gym.make('FrozenLake-v0', is_slippery=True) \n"
                                   "world = world.unwrapped", gym_namespace);

// get the created world
auto world =  boost::python::extract<boost::python::api::object>(gym_namespace["world"]);
```

Observe how we access the environment as an entry in the ```gym_namespace``` we created above. Once we have an instance of the environment, we can query it as shown below

```
auto world_dict = boost::python::extract<boost::python::dict>(world().attr("__dict__"));

auto observation_space = boost::python::extract<boost::python::api::object>(world_dict()["observation_space"]);
std::cout<<"Number of states "<<boost::python::extract<int>(observation_space().attr("__dict__")["n"])          <<std::endl;

auto action_space = boost::python::extract<boost::python::api::object>(world_dict()["action_space"]);
std::cout<<"Number of actions "<<boost::python::extract<int>(action_space().attr("__dict__")["n"])<<std::endl;
```

or execute an action

```
// create an environment
boost::python::exec("observation = world.reset()", gym_namespace);

// the observation
auto observation =  boost::python::extract<int>(gym_namespace["observation"]);
std::cout<<"Observation after reset="<<observation<<std::endl;
```

The full driver program is shown below.

```
#include <boost/python.hpp>
#include <iostream>

int main(){

    try
    {
        std::cout<<"Starting the interpreter..."<<std::endl;

        Py_Initialize();

        std::cout<<"Importing module..."<<std::endl;
        auto gym_module = boost::python::import("gym");
        auto gym_namespace = gym_module.attr("__dict__");

        std::cout<<"Module name "<<boost::python::extract<const char*>(gym_namespace["__name__"])<<std::endl;

        std::cout<<"Creating the environment..."<<std::endl;

        // create an environment
        auto ignored = boost::python::exec("import gym \n"
                                           "world = gym.make('FrozenLake-v0', is_slippery=True) \n"
                                           "world = world.unwrapped", gym_namespace);

        // get the created world
        auto world =  boost::python::extract<boost::python::api::object>(gym_namespace["world"]);

        auto world_dict = boost::python::extract<boost::python::dict>(world().attr("__dict__"));

        // uncomment this to see the attributes
        /*auto keys = boost::python::list(world_dict().keys());

        for(auto i=0; i<boost::python::len(keys); ++i){

            std::cout<<boost::python::extract<std::string>(boost::python::object(keys[i]))()<<std::endl;;
        }*/

        auto observation_space = boost::python::extract<boost::python::api::object>(world_dict()["observation_space"]);
        std::cout<<"Number of states "<<boost::python::extract<int>(observation_space().attr("__dict__")["n"])<<std::endl;

        auto action_space = boost::python::extract<boost::python::api::object>(world_dict()["action_space"]);
        std::cout<<"Number of actions "<<boost::python::extract<int>(action_space().attr("__dict__")["n"])<<std::endl;
        
        // create an environment
        boost::python::exec("observation = world.reset()", gym_namespace);

        // the observation
        auto observation =  boost::python::extract<int>(gym_namespace["observation"]);
        std::cout<<"Observation after reset="<<observation<<std::endl;
    }
    catch(boost::python::error_already_set const &)
    {
        PyErr_Print();
    }

    std::cout<<"Finilize..."<<std::endl;
    return 0;
}

```

Running the program gives the following output

```
Starting the interpreter...
Importing module...
Module name gym
Creating the environment...
Number of states 16
Number of actions 4
Observation after reset=0
Finilize...

```

Although ```boost::python``` handles a lot of the low level details needed for interacting with Python, the above program is rather dense and  for more complicated scenarios, e.g. implementing A2C on an Atari environment, things will definitely get more complicated. One way to handle this is to write own wrappers that hide most of the boilerplate code.


## Buidling the program with CMake

As an aside here is the ```CMakeLists.txt``` to use in order to build the program above

```
CMAKE_MINIMUM_REQUIRED(VERSION 3.6)

SET(SOURCE example_1.cpp)
SET(EXECUTABLE  example_1)

# find Boost
FIND_PACKAGE(Boost 1.65.0 REQUIRED COMPONENTS python system)

if(Boost_FOUND)
  if(Boost_LIBRARY_DIR)
    MESSAGE( STATUS "Boost_LIBRARY_DIR not empty using it: ${Boost_LIBRARY_DIR}" )
  elseif(BOOST_LIBRARYDIR)
      MESSAGE( STATUS "Boost_LIBRARY_DIR empty, but BOOST_LIBRARYDIR is set. Setting Boost_LIBRARY_DIR to: ${BOOST_LIBRARYDIR}" )
      set(Boost_LIBRARY_DIR ${BOOST_LIBRARYDIR})
  elseif(Boost_LIBRARY_DIRS)
      MESSAGE( STATUS "Boost_LIBRARY_DIR empty, but Boost_LIBRARY_DIRS is set. Setting Boost_LIBRARY_DIR to: ${Boost_LIBRARY_DIRS}" )
      set(Boost_LIBRARY_DIR ${Boost_LIBRARY_DIRS})
  elseif(Boost_LIBRARY_DIR_RELEASE)
      MESSAGE( STATUS "Boost_LIBRARY_DIR empty, but Boost_LIBRARY_DIR_RELEASE is set. Setting Boost_LIBRARY_DIR to: ${Boost_LIBRARY_DIR_RELEASE}" )
      set(Boost_LIBRARY_DIR ${Boost_LIBRARY_DIR_RELEASE})
  elseif(Boost_LIBRARY_DIR_DEBUG)
      MESSAGE( STATUS "Boost_LIBRARY_DIR empty, but Boost_LIBRARY_DIR_DEBUG is set. Setting Boost_LIBRARY_DIR to: ${Boost_LIBRARY_DIR_RELEASE}" )
      set(Boost_LIBRARY_DIR ${Boost_LIBRARY_DIR_DEBUG})
  else()
	MESSAGE( WARNING "Boost_LIBRARY_DIR empty, BOOST_LIBRARYDIR empty, Boost_LIBRARY_DIRS empty: might miss libraries at linking" )
  endif()
else()
  MESSAGE( FATAL_ERROR "Boost was not found!")
endif()


INCLUDE_DIRECTORIES(${Boost_INCLUDE_DIRS})

# use c++20 standard
SET(CMAKE_CXX_COMPILER /usr/bin/g++-10)
SET(CMAKE_C_COMPILER /usr/bin/gcc-10)
SET(CMAKE_CXX_STANDARD 20)
SET(CMAKE_CXX_STANDARD_REQUIRED True) 

SET(CMAKE_CXX_FLAGS "-g -Wall -Wextra")
SET(CMAKE_LINKER_FLAGS "-pthread")

# use the Boost link directories
LINK_DIRECTORIES(${Boost_LIBRARY_DIR})

# this may be different...
LINK_DIRECTORIES(/usr/lib/python3.8/config-3.8-x86_64-linux-gnu/)

ADD_EXECUTABLE(${EXECUTABLE} ${SOURCE})


TARGET_LINK_LIBRARIES(${EXECUTABLE} python3.8)
TARGET_LINK_LIBRARIES(${EXECUTABLE} boost_python38)
TARGET_LINK_LIBRARIES(${EXECUTABLE} boost_system)

```